In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
from glob import glob

def extract_slices_from_folder(input_folder, output_base_dir):
    nii_files = glob(os.path.join(input_folder, '*.nii.gz'))

    for nii_file in nii_files:
        img = nib.load(nii_file)
        data = img.get_fdata()

        base_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file))[0])[0]

        folders = [f"{base_name}_x", f"{base_name}_y", f"{base_name}_z"]
        for folder in folders:
            os.makedirs(os.path.join(output_base_dir, folder), exist_ok=True)

        for axis in range(3):
            for i in range(224):  # Always generate 224 images

                if axis == 0:
                    slice_data = np.zeros((data.shape[1], data.shape[2], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[i-1, :, :] if i-1 < data.shape[0] else 0
                    slice_data[:, :, 1] = data[i, :, :] if i < data.shape[0] else 0
                    if i < data.shape[0] - 1:
                        slice_data[:, :, 2] = data[i+1, :, :] if i+1 < data.shape[0] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_x", f"{base_name}_x_{i:03d}.jpg")

                elif axis == 1:
                    slice_data = np.zeros((data.shape[0], data.shape[2], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[:, i-1, :] if i-1 < data.shape[1] else 0
                    slice_data[:, :, 1] = data[:, i, :] if i < data.shape[1] else 0
                    if i < data.shape[1] - 1:
                        slice_data[:, :, 2] = data[:, i+1, :] if i+1 < data.shape[1] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_y", f"{base_name}_y_{i:03d}.jpg")
                    
                else:
                    slice_data = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[:, :, i-1] if i-1 < data.shape[2] else 0
                    slice_data[:, :, 1] = data[:, :, i] if i < data.shape[2] else 0
                    if i < data.shape[2] - 1:
                        slice_data[:, :, 2] = data[:, :, i+1] if i+1 < data.shape[2] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_z", f"{base_name}_z_{i:03d}.jpg")

                # Set -1500 as background
                slice_data[slice_data <= -1500] = -1500
                # Normalize the data to 0-255 range
                slice_data = ((slice_data - slice_data.min()) / (slice_data.max() - slice_data.min() + 1e-8) * 255).astype(np.uint8)
                
                # Create and save the image
                img = Image.fromarray(slice_data)
                img.save(out_file)

        print(f"224 slices extracted from {base_name} and saved in folders: {', '.join(folders)}")

    print("All .nii.gz files processed.")

# Usage example
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
output_base_dir = r"C:\Users\acer\Desktop\Data_0\Nii_toJPG_3layers"
extract_slices_from_folder(input_folder, output_base_dir)